In [21]:
import pandas as pd
df = pd.read_csv("dataset/TC-dataset.csv", sep="\t",index_col=0)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 471910 entries, 0 to 541909
Data columns (total 8 columns):
BasketID           471910 non-null object
BasketDate         471910 non-null object
Sale               471910 non-null float64
CustomerID         406830 non-null float64
CustomerCountry    471910 non-null object
ProdID             471910 non-null object
ProdDescr          471157 non-null object
Qta                471910 non-null int64
dtypes: float64(2), int64(1), object(5)
memory usage: 32.4+ MB


## Data Selection

In questa sezione a partire dai dati originali si effettua una selezione dei dati in modo tale da eliminare tutti quelli semanticamente errati. Queste eliminazioni provengono da alcune assunzioni effettuate sui dati e sulla loro provenienza

### Provenienza dei dati
I dati provengono dal dataset https://archive.ics.uci.edu/ml/datasets/Online+Retail Nella fase di data semantics abbiamo già precisato alcune informazioni:
Si tratta di un sito e-commerce Inglese. Vende principalmente regali per tutte le occasioni e molti dei clienti sono grossisti. Nella colonna Sale è presente il prezzo per singolo articolo quindi equivale a quantità=1. Il prezzo è in sterline. Secondo la descrizione del dataset quando in basket id abbiamo una C iniziale si tratta di ordini cancellati.


### Trattamento Ordini cancellati
Per quanto ci riguarda gli ordini cancellati non sono significativi per noi per caratterizzare i clienti. Di conseguenza possiamo eliminarli. Tutti gli ordini da cancellare hanno anche quantità<=0

In [22]:
print("Ordini da cancellare con \"C\": ",len(df[df["BasketID"].str.contains("C")]))
print("Ordini da cancellare con Qta<=0: ",len(df[df["Qta"]<=0]))
print("Record da cancellare con \"C\" con quantità<=0: "+str(len(df[(df["Qta"]<=0) & (df["BasketID"].str.contains("C"))])))

Ordini da cancellare con "C":  9084
Ordini da cancellare con Qta<=0:  9752
Record da cancellare con "C" con quantità<=0: 9084


Ogni ordine cancellato potrebbe averne un corrispondente positivo quindi prima di eliminarli verifichiamo quindi se troviamo degli ordini opposti, prediamo tutti quelli negativi e verifichiamo.

In [23]:
list_baskC=df[df["BasketID"].str.contains("C")]
count=0

for index, row in list_baskC.iterrows():

    local_search=df[(df["CustomerID"]==row['CustomerID'])&(df["Sale"]==row['Sale'])&(df["Qta"]==-row['Qta'])&(df["ProdID"]==row['ProdID'])]

    if len(local_search)>0:# In questo caso vuol dire che ci sono due ordini opposti
        count+=1
        ##print(index)
        ##print(df.loc[index])
        ##print(local_search.index[0])
        ##print(df.loc[local_search.index[0]])
        ##print("_________________________")
        df.drop([index,local_search.index[0]], inplace=True)
        
        
        
print("Record cancellati: ",2*count)
print("Nuova grandezza del dataset: ",len(df))

Record cancellati:  6144
Nuova grandezza del dataset:  465766


Nell'analisi non abbiamo considerato che non per forza un ordine cacellato ha la stessa quantità, potrebbe essere su un ordine di 50 elementi solo 20 per esempio siano stati cancellati. Tuttavia abbiamo semplificato il problema per evitare analisi troppe complicate. Magari con lo sviluppo del progetto potremo valutare di migliorare questa analisi. Vediamo quanti record rimangono:

In [24]:
print("Record ancora da eliminare: ",len(df[df["BasketID"].str.contains("C"]))


Record ancora da eliminare:  6680


Possiamo prendere i rimanenti e cercare nel dataset se c'è un altro ordine positivo con una quantità maggiore del valore assoluto in modo tale da elimiare un po' di quantità. Quindi rispetto a prima rimangono da fare i record in cui il valore assoluto della quantità negativa non ha un record >=

In [25]:
remaining_order=df[df["BasketID"].str.contains("C")]
count=0

for index,row in remaining_order.iterrows():
    transaction_retrived=df[(df["CustomerID"]==row["CustomerID"])&(df["ProdID"]==row["ProdID"])&(df["Sale"]==row["Sale"])&(df["Qta"]>-row["Qta"])].sort_values(by=['BasketDate'])
    if len(transaction_retrived)>0:
        df.drop([index], inplace=True)
        transaction_retrived.Qta.iloc[0]=transaction_retrived.Qta.iloc[0]-row["Qta"]
        count+=1
print("Record eliminati: ",count)

Record eliminati:  4445


In [26]:
len(df[df["BasketID"].str.contains("C")])

1567

Oltre a quelli opposti verifichiamo che non ci siano altri ordini cancellati. In questo caso li eliminamo comunque perchè potrebbero essere errori, anche se non hanno un corrispettivo positivo. Oppure potrebbero essere ordini cancellati corrispondenti a ordini fatti prima del dicembre 2010.

In [27]:
print("Record da eliminare: ",len(df[df["BasketID"].str.contains("C")]))
df=df[df["BasketID"].str.contains("C") == False]

Record da eliminare:  1567


### Quantità negative con ordini non cancellati
Abbiamo già constatato che gli ordini con la C sono cancellati e abbiamo già deciso di cancellarli. Consideriamo ora quelli con solo le quantità negative.

In [28]:
print("Record con quantità <=0: ", len(df[df["Qta"]<=0]))
print("Record con quantità <=0 e Sale<=0: ", len(df[(df["Qta"]<=0) & (df["Sale"]<=0)]))
print(df[df["Qta"]<=0].ProdDescr.unique())


Record con quantità <=0:  668
Record con quantità <=0 e Sale<=0:  668
[nan 'check' 'Dotcom sales' 'reverse 21/5/10 adjustment'
 'mouldy, thrown away.' '?' 'label mix up' 'samples/damages' 'thrown away'
 'damages' 'showroom' 'wrongly sold as sets' 'dotcom sold sets'
 'Amazon sold sets' 'wrongly sold sets' '?sold as sets?' 'damages/display'
 'damaged stock' 'damages?' 're dotcom quick fix.' 'sold in set?'
 'POSSIBLE DAMAGES OR LOST?' 'damaged' 'Damaged' 'DAMAGED' 'Missing'
 'wrong code?' 'crushed' 'damages/credits from ASOS.'
 'Not rcvd in 10/11/2010 delivery' 'Thrown away-rusty' 'damages/dotcom?'
 'smashed' 'reverse previous adjustment'
 'incorrectly credited C550456 see 47' 'wet damaged' 'Water damaged'
 'missing' 'sold as set on dotcom' 'mix up with c' 'ebay'
 'Sold as 1 on dotcom' 'taig adjust no stock' '?display?' '??'
 'OOPS ! adjustment' 'Dagamed' 'historic computer difference?....se'
 'incorrect stock entry.' 'wrongly coded-23343' 'stock check' 'WET/MOULDY'
 'Wet pallet-thrown aw

Abbiamo solamente 668 transazioni in cui abbiamo la quantità negativa e non appartengono a ordini cancellati. Inoltre in questi 668 tutti hanno sale<=0.  Dalle descrizioni fornite la maggior parte sono quindi articoli persi oppure danneggiati o eventuali errori. 
Eliminare anche questi record, in futuro possiamo pensare a qualche indicatore che possa tenere conto di questi articoli. In ogni caso sono solo 668 elementi.

In [29]:
df=df[df["Qta"]>0]

### Sale <=0 e quantità >=0
Abbiamo trattato le quantità <=0. Altri valori particolari sono quelli relativi al Sale. In particolare ci sono alcuni sale che sono <=0.

In [30]:
print("Sale<=0: ",len((df[df["Sale"]<=0])))
print("Sale<0: ",len((df[df["Sale"]<0]))," record\n", df[df["Sale"]<0])

Sale<=0:  613
Sale<0:  2  record
        BasketID           BasketDate      Sale  CustomerID CustomerCountry  \
299983  A563186  2011-08-12 14:51:00 -11062.06         NaN  United Kingdom   
299984  A563187  2011-08-12 14:52:00 -11062.06         NaN  United Kingdom   

       ProdID        ProdDescr  Qta  
299983      B  Adjust bad debt    1  
299984      B  Adjust bad debt    1  


Esistono solo 2 record con sale < 0 che eliminati in quanto non rappresentano una informazione relativa ai customer ma sono relativi all'azienda.

611 Record hanno sia il sale > 0 e la quantità > 0, essendo il nostro clustering fatto sui clienti e non sui prodotti potrebbe fuoriviante avere alcuni record con sale = 0.
Potrebbero essere omaggi per esempio e non avrebbe senso considerarli.
Alcuni con il sale = 0 hanno quantità molto alte anche di 300 unità, in questo caso sarebbe anche difficile considerarli omaggi a meno che non siano per grossisti.

Dato che stiamo performando una analisi sui customer trovo poco significativo considerare gli ordini "omaggio" che sono una registrazione che viene fatta dall'azienda per tenerne traccia

Altra possibilità fare il clustering con questi e vedere se effettivamente ci deviano il clustering, sono comunque pochi record.

In [31]:
df=df[df["Sale"]>=0]